In [3]:
import openai

client = openai.OpenAI()

content = """다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.

문장: \"나는 빨간사과가 좋더라\""""

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "user", "content": content}
  ]
)

print(response.choices[0].message.content)


{
    "color": "빨간(red)"
}


In [4]:
content = """다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.

문장: \"아침에는 인간적으로 커피 한 잔 주고 뭐라 하자\""""

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "user", "content": content}
  ]
)

print(response.choices[0].message.content)


{
  "color": "인간적으로",
  "translated": "humanely"
}


In [5]:
schema = {
  "$schema": "http://json-schema.org/draft-07/schema",
  "type": "object",
  "properties": {
    "color": {
      "type": "string",
    }
  },
  "required": ["color"]
}

content = """다음 문장에서 색깔 정보가 있으면 그것을 추출하여 영어로 번역하고 json 포맷으로 만들어서 출력해라.

문장: '나는 빨간 사과가 좋더라'

json schema 정보는 다음과 같다: {schema}

색깔 정보가 없으면 no color로 대신해라
"""

response = client.chat.completions.create(
  model="gpt-3.5-turbo-0125",
  messages=[
    {"role": "user", "content": content}
  ]
)

print(response.choices[0].message.content)


{
    "color": "빨간(red)"
}


In [6]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": '나는 노란색이 좋더라'}
  ],
  functions=[
    {
      "name": "color_extractor",
      "description": "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력함",
      "parameters": {
        "type": "object",
        "properties": {
          "color": {
            "type": "string",
            "description": "색깔 정보를 빼내와서 그에 해당하는 영어 단어를 출력함"
          }
        },
        "required": ["color"]
      }
    }
  ]
)

print(response.choices[0].message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"color":"노란색"}', name='color_extractor'), tool_calls=None)


In [7]:
print(response.choices[0].message.function_call.arguments)

{"color":"노란색"}


In [8]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": '퇴근했는데 업무 톡 날리는 양심 없는 인간'}
  ],
  functions=[
    {
      "name": "color_extractor",
      "description": "사용자의 선호 색깔을 찾아서 영어로 번역하고 영어 단어를 출력함",
      "parameters": {
        "type": "object",
        "properties": {
          "color": {
            "type": "string",
            "description": "색깔 정보를 빼내와서 그에 해당하는 영어 단어를 출력함"
          }
        },
        "required": ["color"]
      }
    }
  ]
)

print(response.choices[0].message)


ChatCompletionMessage(content='퇴근 후에도 업무 관련 연락을 받는 건 정말 스트레스죠. 휴식 시간은 정말 소중한데, 그런 경우에는 "현재 퇴근 중이라 후에 확인하겠습니다" 라고 간단히 답장하는 것도 좋을 것 같아요. 이런 상황에서 어떻게 대응하면 좋을지 도움이 필요하시면 말씀해 주세요!', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [9]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": '아침에 세 개 저녁에 네 개'}
  ],
  functions=[
    {
      "name": "apple_add",
      "description": "사과 숫자를 더함",
      "parameters": {
        "type": "object",
        "properties": {
          "quantity1": {
            "type": "integer",
            "description": "첫번째 숫자"
          },
          "quantity2": {
            "type": "integer",
            "description": "두번째 숫자"
          }
        },
        "required": ["quantity1", "quantity2"]
      }
    }
  ],
  function_call="auto"
)

print(response.choices[0].message)


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"quantity1":3,"quantity2":4}', name='apple_add'), tool_calls=None)


In [10]:
def apple_add(q1, q2):
  return str(q1 + q2)

message = response.choices[0].message

print(response.choices[0].message.function_call)




FunctionCall(arguments='{"quantity1":3,"quantity2":4}', name='apple_add')


In [12]:
from duckpy import Client

duckduckgo_client = Client()

def duck_search(query) -> str:
  output = duckduckgo_client.search(query)
  return str(output)

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  temperature=0,
  functions=[
    {
      "name": "duck_search",
      "description": "'Used to search online",
      "parameters": {
        "type": "object",
        "properties": {
          "query": {
            "type": "string",
            "description": "Translate the Korean content into English input query"
          }
        },
        "required": ["query"]
      }
    }
  ],
  messages=[
    {"role": "system", "content": "You must use `duck_search` function to get information"},
    {"role": "user", "content": "조지 클루니 생일이 언제야?"}
  ]
)

print(completion)



ChatCompletion(id='chatcmpl-BOPEvMJymuKivUNuEdV0sTTYrhKjc', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{"query":"George Clooney birthday"}', name='duck_search'), tool_calls=None))], created=1745156593, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_0392822090', usage=CompletionUsage(completion_tokens=19, prompt_tokens=76, total_tokens=95, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [15]:
message = completion.choices[0].message

import ast

if message.function_call:
  function_name = message.function_call.name
  args = ast.literal_eval(message.function_call.arguments)

  function_response = duck_search(
    query = args.get("query")
  )

  print(function_response)

  completion_final = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "user", "content": '조지클루니 생일이 언제야'},
      message,
      {
        "role": "function",
        "name": function_name,
        "content": function_response
      }
    ]
  )

  print(completion_final.choices[0].message.content)
  

{'query': 'George Clooney birthday'}
[{'title': 'George Clooney - Wikipedia', 'description': 'George Timothy Clooney[2] (born May 6, 1961) is an American actor, filmmaker, and philanthropist. Known for his leading man roles on screen in both blockbuster and independent films, Clooney has received numerous accolades, including two Academy Awards, a BAFTA Award and four Golden Globe Awards as well as nominations for three Primetime Emmy Awards. His honors include the Cecil B. DeMille ...', 'url': 'https://en.wikipedia.org/wiki/George_Clooney'}, {'title': 'George Clooney: Biography, Actor, Oscar Winner', 'description': "Oscar winner George Clooney is known for 'Syriana\ufeff,' 'The Descendants,' and the 'Ocean's' franchise. Read about his movies, TV shows, kids, wife, and more.", 'url': 'https://www.biography.com/actors/george-clooney'}, {'title': 'George Clooney - Age, Family, Bio | Famous Birthdays', 'description': "George Clooney is a movie actor, director, writer, and producer who was

In [16]:
example_user_input = "런던 날씨가 안 좋다면서"

completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": example_user_input}
  ],
  functions=[
    {
      "name": "get_different_responses",
      "description": "답변을 여러가지 말투로 하기",
      "parameters": {
        "type": "object",
        "properties": {
          "response_up": {
            "type": "string",
            "description": "답변을 높임말로 바꿈"
          },
          "response_down": {
            "type": "string",
            "description": "답변을 반말루 바꿈"
          }
        },
        "required": ["response_up", "response_down"]
      }
    }
  ],
  function_call={"name": "get_different_responses"}
)

reply_content = completion.choices[0].message

print(reply_content.function_call.arguments)

{"response_up":"런던의 날씨가 좋지 않다고 하더라고요. 비가 오거나 흐린 날이 많아서 외출할 때 주의가 필요할 것 같습니다.","response_down":"런던 날씨가 별로래. 비 오거나 흐린 날이 많아서 나가기 조심해야 할듯."}
